In [2]:
pip install openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 24.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.2 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803319 sha256=07651c5e69b33225df3192817d1683691ed9558091f8554d3cba381daee421d6
  Stored in directory: /root/.cache/pip/wheels/dd/4a/1f/d1c4bf3b9133c8168fe617ed979cab7b14fe381d059ffb9d83
Successfully built openai-whisper


In [3]:
pip install pyannote.audio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 12.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.7/898.7 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.9/125.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 824.8/824.8 kB 43.9 MB/s eta 0:00:00

In [1]:
import whisper
from transformers import pipeline
from datetime import datetime
import os
import shutil
from pyannote.audio import Pipeline
import torch
import wave
import contextlib

class CallProcessor:
    def __init__(self, base_dir="call_records", auth_token=None):
        self.base_dir = base_dir
        self.audio_dir = os.path.join(base_dir, "audio")
        self.summary_dir = os.path.join(base_dir, "summaries")

        # Initialize Whisper for transcription
        self.transcriber = whisper.load_model("base")
        # Initialize summarization pipeline
        self.summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
        # Initialize speaker diarization pipeline
        if auth_token:
            self.diarization = Pipeline.from_pretrained(
                "pyannote/speaker-diarization@2.1",
                use_auth_token=auth_token
            )
            if torch.cuda.is_available():
                self.diarization = self.diarization.to(torch.device("cuda"))
        else:
            print("Warning: No auth token provided for speaker diarization")
            self.diarization = None

        # Create directories if they don't exist
        for directory in [self.audio_dir, self.summary_dir]:
            if not os.path.exists(directory):
                os.makedirs(directory)

    def get_audio_duration(self, audio_path):
        """Get duration of audio file in seconds"""
        with contextlib.closing(wave.open(audio_path, 'r')) as f:
            frames = f.getnframes()
            rate = f.getframerate()
            duration = frames / float(rate)
            return duration

    def diarize_audio(self, audio_path):
        """
        Perform speaker diarization on audio file
        Returns list of segments with speaker labels and timestamps
        """
        if not self.diarization:
            print("Speaker diarization not available - missing auth token")
            return None

        try:
            # Run diarization
            diarization = self.diarization(audio_path)

            # Convert to list of segments
            segments = []
            for turn, _, speaker in diarization.itertracks(yield_label=True):
                segments.append({
                    'speaker': speaker,
                    'start': turn.start,
                    'end': turn.end
                })
            return segments
        except Exception as e:
            print(f"Error during diarization: {e}")
            return None

    def transcribe_with_speakers(self, audio_path, diarization_segments):
        """
        Transcribe audio with speaker labels
        """
        try:
            # Get full transcription
            result = self.transcriber.transcribe(audio_path)
            full_text = result["text"]

            if not diarization_segments:
                return full_text

            # Break transcription into segments based on timestamps
            segments = result["segments"]

            # Match transcription segments with speaker labels
            transcription_with_speakers = []
            for segment in segments:
                # Find matching speaker segment
                segment_mid_time = (segment['start'] + segment['end']) / 2
                matching_speaker = None

                for speaker_segment in diarization_segments:
                    if (speaker_segment['start'] <= segment_mid_time and
                        speaker_segment['end'] >= segment_mid_time):
                        matching_speaker = speaker_segment['speaker']
                        break

                # Format segment with speaker label
                speaker_label = matching_speaker if matching_speaker else "Unknown Speaker"
                transcription_with_speakers.append(
                    f"[{speaker_label}]: {segment['text']}"
                )

            return "\n".join(transcription_with_speakers)

        except Exception as e:
            print(f"Error during transcription: {e}")
            return None

    def save_audio(self, audio_file_path):
        """
        Save a copy of the audio file with timestamp
        """
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        file_extension = os.path.splitext(audio_file_path)[1]
        new_filename = f"call_{timestamp}{file_extension}"
        new_path = os.path.join(self.audio_dir, new_filename)

        try:
            shutil.copy2(audio_file_path, new_path)
            return new_path, timestamp
        except Exception as e:
            print(f"Error saving audio file: {e}")
            return None, None

    def generate_summary(self, text):
        """
        Generate a summary of the transcribed text
        """
        try:
            max_chunk_length = 1024
            chunks = [text[i:i + max_chunk_length] for i in range(0, len(text), max_chunk_length)]

            summaries = []
            for chunk in chunks:
                summary = self.summarizer(chunk, max_length=130, min_length=30, do_sample=False)
                summaries.append(summary[0]['summary_text'])

            return " ".join(summaries)
        except Exception as e:
            print(f"Error during summarization: {e}")
            return None

    def format_email(self, transcription, summary, audio_path, speaker_count=None, call_metadata=None):
        """
        Format the email with transcription and summary
        """
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M")

        email_body = f"""
Subject: Call Summary Report - {timestamp}

AUDIO FILE LOCATION:
{audio_path}

{"NUMBER OF SPEAKERS DETECTED: " + str(speaker_count) if speaker_count else ""}

SUMMARY:
{summary}

FULL TRANSCRIPTION (WITH SPEAKERS):
{transcription}

{'CALL METADATA:' if call_metadata else ''}
{call_metadata if call_metadata else ''}
"""
        return email_body

    def save_summary(self, email_content, timestamp):
        """
        Save the email content to a text file
        """
        filename = f"call_summary_{timestamp}.txt"
        filepath = os.path.join(self.summary_dir, filename)

        try:
            with open(filepath, 'w', encoding='utf-8') as f:
                f.write(email_content)
            print(f"Summary saved to: {filepath}")
            return filepath
        except Exception as e:
            print(f"Error saving summary: {e}")
            return None

def process_call(audio_file_path, auth_token, base_dir="call_records"):
    """
    Main function to process a call recording
    """
    processor = CallProcessor(base_dir, auth_token)

    # Step 1: Save audio file
    print("Saving audio file...")
    saved_audio_path, timestamp = processor.save_audio(audio_file_path)
    if not saved_audio_path:
        return False

    # Step 2: Perform speaker diarization
    print("Detecting speakers...")
    diarization_segments = processor.diarize_audio(saved_audio_path)
    speaker_count = len(set(segment['speaker'] for segment in diarization_segments)) if diarization_segments else None

    # Step 3: Transcribe with speaker labels
    print("Transcribing audio...")
    transcription = processor.transcribe_with_speakers(saved_audio_path, diarization_segments)
    if not transcription:
        return False

    # Step 4: Summarize
    print("Generating summary...")
    summary = processor.generate_summary(transcription)
    if not summary:
        return False

    # Step 5: Format email and save
    print("Formatting and saving summary...")
    email_content = processor.format_email(
        transcription=transcription,
        summary=summary,
        audio_path=saved_audio_path,
        speaker_count=speaker_count,
        call_metadata={
            "Audio File": os.path.basename(saved_audio_path),
            "Date": datetime.now().strftime("%Y-%m-%d %H:%M"),
            "Duration": f"{processor.get_audio_duration(saved_audio_path):.2f} seconds",
            "Number of Speakers": speaker_count if speaker_count else "Unknown"
        }
    )

    # Save summary
    saved_summary = processor.save_summary(email_content, timestamp)
    return saved_summary is not None

In [2]:
auth_token = "EXAMPLETOKEN"
audio_file = "/content/phonecall.wav"
success = process_call(audio_file, auth_token)

100%|███████████████████████████████████████| 139M/139M [00:02<00:00, 54.5MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

label_encoder.txt:   0%|          | 0.00/129k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/speechbrain/utils/checkpoints.py:200: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(path, map_location=device

Saving audio file...
Detecting speakers...


/usr/local/lib/python3.10/dist-packages/pyannote/audio/utils/reproducibility.py:74: ReproducibilityWarning: TensorFloat-32 (TF32) has been disabled as it might lead to reproducibility issues and lower accuracy.
It can be re-enabled by calling
   >>> import torch
   >>> torch.backends.cuda.matmul.allow_tf32 = True
   >>> torch.backends.cudnn.allow_tf32 = True
See https://github.com/pyannote/pyannote-audio/issues/1370 for more details.

  warnings.warn(


Transcribing audio...
Generating summary...
Formatting and saving summary...
Summary saved to: call_records/summaries/call_summary_20250105_214955.txt
